## W2-CRISPR-cBEST workflow


Base-editing with CRISPR-BEST is a DSB-free method that uses a single sgRNA to target a specific genomic location with single-nucleotide–resolution(Figure 3A) which can enable in-vivo protein engineering or as it is often used, for introducing early stop-codons10,12. It can be used to introduce targeted point mutations in the genome of Streptomyces, facilitating studies on gene function and protein engineering. To get started quickly users can download the pCRISPR-cBEST plasmid file, the genome file for S.coelicor (A3) and streptoAIM will generate your plasmids based on the genes you chose to target (Figure 3, S5). 

In [1]:
import sys
import os

# Ensure the src directory is in the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import os
import pandas as pd
from pydna.dseqrecord import Dseqrecord
from datetime import datetime

from streptocad.sequence_loading.sequence_loading import  load_and_process_plasmid, load_and_process_genome_sequences, annotate_dseqrecord, check_and_convert_input, process_specified_gene_sequences_from_record
from streptocad.utils import polymerase_dict,ProjectDirectory, extract_metadata_to_dataframe
from streptocad.crispr.guideRNAcas3_9_12 import extract_sgRNAs, SgRNAargs
from streptocad.cloning.ssDNA_bridging import assemble_plasmids_by_ssDNA_bridging, make_ssDNA_oligos
from streptocad.crispr.crispr_best import identify_base_editing_sites, filter_sgrnas_for_base_editing, process_base_editing
from streptocad.cloning.plasmid_processing import annotate_plasmid_with_sgrnas
from streptocad.primers.primer_generation import find_best_check_primers_from_genome, create_idt_order_dataframe, primers_to_IDT

In [2]:
from pydna.dseqrecord import Dseqrecord

def find_locus_tags(record: Dseqrecord, max_tags: int = None):
    """
    Iterate through a Dseqrecord and return a list of unique locus tags found in its features,
    stopping after `max_tags` tags if specified.

    :param record: pydna.dseqrecord.Dseqrecord object
    :param max_tags: maximum number of locus tags to collect (None for no limit)
    :return: list of locus_tag strings
    """
    tags = []
    seen = set()
    count = 0

    # Ensure the record has features
    if not hasattr(record, 'features'):
        return tags

    for feature in record.features:
        if max_tags is not None and count >= max_tags:
            break

        qualifiers = getattr(feature, 'qualifiers', {})

        for tag in qualifiers.get('locus_tag', []):
            if max_tags is not None and count >= max_tags:
                break
            # only append new tags
            if tag not in seen:
                seen.add(tag)
                tags.append(tag)
                count += 1

        if max_tags is not None and count >= max_tags:
            break

    return tags


## INPUT

In [3]:
# Inputs
# 1 Add genome of choice (genbank, fasta)
path_to_genome = '../../../../data/genomes/Streptomyces_coelicolor_A3_chromosome.gb'
genome = load_and_process_genome_sequences(path_to_genome)[0]

# 2 Add plasmid 
path_to_plasmid = '../../../../data/plasmids/ pCRISPR-cBEST Sequences.gbk'
clean_plasmid = load_and_process_plasmid(path_to_plasmid)

# 3 Choose genes to knock out (list)
locus_tags = find_locus_tags(genome, 1000)
genes_to_KO = locus_tags

#### Advanced settings ####
# 3 Choose polymerase and target melting temperature FOR CHECKING PRIMERS
chosen_polymerase = polymerase_dict['Phusion High-Fidelity DNA Polymerase (GC Buffer)']

melting_temperature = 65
primer_concentration = 0.4 
primer_number_increment = 1
flanking_region_number = 500

# 4 Filtering metrics for sgRNAs
gc_upper = 0.99
gc_lower = 0.01
off_target_seed = 13
off_target_upper = 10
cas_type='cas9'
number_of_sgRNAs_per_group = 1
only_stop_codons = True
flanking_region = 500

# 6 Choose overlapping sequences for our plasmid we can use the following
#As per the article **"CRISPR–Cas9, CRISPRi and CRISPR-BEST-mediated genetic manipulation in streptomycetes"** we need the following oligoes: 
#CGGTTGGTAGGATCGACGGC **-N20-** GTTTTAGAGCTAGAAATAGC
up_homology = Dseqrecord('cggttggtaggatcgacggc') # V1 homology : CGGTTGGTAGGATCGACGGC
dw_homology = Dseqrecord('tatgtcctccgagaccggcc') # V1 homology : GTTTTAGAGCTAGAAATAGC

# Computation

In [4]:
from teemi.design.fetch_sequences import read_genbank_files     
sequences = read_genbank_files(path_to_plasmid)
sequences


[SeqRecord(seq=Seq('GGCCGTCGACGACGACGCGTCCGCCTGCCTGCTTTTGGCCGGCGGGCCGGGCCG...CCA'), id='pCRISPR-cBEST(#125689)', name='pCRISPR-cBEST', description='C to T base editor for actinomycetes', dbxrefs=[])]

In [5]:
target_dict, genes_to_KO, annotation_input = check_and_convert_input(genes_to_KO)

print(annotation_input)
if annotation_input == True:
    genome = annotate_dseqrecord(genome, target_dict)



False


In [6]:
len(genes_to_KO)

1000

We need to check the input genes so we can find sgRNAs. 

In [7]:

# Initialize SgRNAargs with desired parameters
args = SgRNAargs(genome, 
                genes_to_KO,
                step=['find', 'filter'],
                gc_upper = gc_upper,
                gc_lower = gc_lower,
                off_target_seed = off_target_seed,
                off_target_upper = off_target_upper,
                cas_type='cas9',
                )

sgrna_df = extract_sgRNAs(args)
sgrna_df

sgRNA generated were outside the designated border in SCO0001. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0001. To incorporate this extent borders. Skipping to next locus tag.
Pam was found outside designated locus_tag: SCO0002. To incorporate this extent borders. Skipping to next locus tag.
Pam was found outside designated locus_tag: SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRN

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count
4085,NC_003888.3,SCO0022,22156,-1,1,127,0.65,CGG,CACGAGAACCCCGAGCAGTG,ACCCCGAGCAGTG,-1
4122,NC_003888.3,SCO0022,22156,-1,-1,270,0.65,CGG,GGCCAGGTTGGGCCAGGTTT,TTGGGCCAGGTTT,-1
4123,NC_003888.3,SCO0022,22156,-1,-1,264,0.75,AGG,GAGTGGGGCCAGGTTGGGCC,GCCAGGTTGGGCC,-1
4124,NC_003888.3,SCO0022,22156,-1,-1,259,0.65,GGG,TCAGCGAGTGGGGCCAGGTT,GTGGGGCCAGGTT,-1
4096,NC_003888.3,SCO0022,22156,-1,1,275,0.70,TGG,CGCCGAAACCTGGCCCAACC,ACCTGGCCCAACC,-1
...,...,...,...,...,...,...,...,...,...,...,...
49461,NC_003888.3,SCO0281,272466,-1,1,224,0.85,AGG,CGGCGTCGCAGCGGGCCAGG,GCAGCGGGCCAGG,10
58645,NC_003888.3,SCO0319,319499,-1,1,311,0.90,AGG,TCCCGTGCCCGGCCCCGGCC,CCCGGCCCCGGCC,10
85527,NC_003888.3,SCO0428,445611,1,1,342,0.75,CGG,GCGTTCACCGCGGTCCTGCG,CCGCGGTCCTGCG,10
60309,NC_003888.3,SCO0324,325238,-1,-1,23,0.95,TGG,CCGCGCCGCGCTGGCCGCCC,GCGCTGGCCGCCC,10


In [8]:
value_counts = sgrna_df['locus_tag'].value_counts()
value_counts

locus_tag
SCO0492    2522
SCO0127    1768
SCO0126    1581
SCO0216     839
SCO0432     832
           ... 
SCO0060      18
SCO0228      17
SCO0038      14
SCO0045      14
SCO0056      12
Name: count, Length: 979, dtype: int64

In [9]:
len(genome.features)

25824

In [10]:
# Load gene sequences from the genome
gene_sequences = process_specified_gene_sequences_from_record(genome, genes_to_KO)
genes_to_KO_dict = {locus_tag: gene_sequences[locus_tag] for locus_tag in genes_to_KO if locus_tag in gene_sequences}
print(genes_to_KO_dict)
# Identify and annotate base editing sites
sgrna_df_with_editing = identify_base_editing_sites(sgrna_df)

# filter out only sgRNAs that result in base-editing
filtered_sgrna_df_for_base_editing = filter_sgrnas_for_base_editing(sgrna_df_with_editing)
filtered_sgrna_df_for_base_editing

...
{'SCO0001': Dseq(-678)
ATGA..ATAG
TACT..TATC, 'SCO0002': Dseq(-2562)
ATGG..GTAG
TACC..CATC, 'SCO0003': Dseq(-2352)
GTGA..CTGA
CACT..GACT, 'SCO0004': Dseq(-948)
GTGC..GTAA
CACG..CATT, 'SCO0005': Dseq(-2121)
GTGG..GTGA
CACC..CACT, 'SCO0006': Dseq(-1107)
GTGA..GTGA
CACT..CACT, 'SCO0007': Dseq(-447)
GTGT..ATGA
CACA..TACT, 'SCO0008': Dseq(-351)
GTGG..GTAG
CACC..CATC, 'SCO0009': Dseq(-336)
TTGT..GTGA
AACA..CACT, 'SCO0010': Dseq(-1092)
GTGA..GTGA
CACT..CACT, 'SCO0011': Dseq(-810)
GTGC..GTGA
CACG..CACT, 'SCO0012': Dseq(-411)
GTGG..CTGA
CACC..GACT, 'SCO0013': Dseq(-291)
GTGC..GTGA
CACG..CACT, 'SCO0014': Dseq(-279)
ATGG..CTGA
TACC..GACT, 'SCO0015': Dseq(-1233)
ATGT..GTGA
TACA..CACT, 'SCO0016': Dseq(-204)
GTGA..GTGA
CACT..CACT, 'SCO0017': Dseq(-345)
GTGA..CTGA
CACT..GACT, 'SCO0018': Dseq(-867)
ATGG..CTGA
TACC..GACT, 'SCO0019': Dseq(-255)
ATGT..GTAG
TACA..CATC, 'SCO0020': Dseq(-1647)
ATGG..ATGA
TACC..TACT, 'SCO0021': Dseq(-594)
ATGA..GTAG
TACT..CATC, 'SCO0022': Dseq(-423)
CGGC..CTGA
GCCG..GACT

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,editing_context,editable_cytosines
4085,NC_003888.3,SCO0022,22156,-1,1,127,0.65,CGG,CACGAGAACCCCGAGCAGTG,ACCCCGAGCAGTG,-1,"0,0","9,10"
4122,NC_003888.3,SCO0022,22156,-1,-1,270,0.65,CGG,GGCCAGGTTGGGCCAGGTTT,TTGGGCCAGGTTT,-1,0,4
4123,NC_003888.3,SCO0022,22156,-1,-1,264,0.75,AGG,GAGTGGGGCCAGGTTGGGCC,GCCAGGTTGGGCC,-1,"1,0","9,10"
4124,NC_003888.3,SCO0022,22156,-1,-1,259,0.65,GGG,TCAGCGAGTGGGGCCAGGTT,GTGGGGCCAGGTT,-1,1,5
4096,NC_003888.3,SCO0022,22156,-1,1,275,0.70,TGG,CGCCGAAACCTGGCCCAACC,ACCTGGCCCAACC,-1,"0,0,0","4,9,10"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49461,NC_003888.3,SCO0281,272466,-1,1,224,0.85,AGG,CGGCGTCGCAGCGGGCCAGG,GCAGCGGGCCAGG,10,"1,0,1","4,7,9"
58645,NC_003888.3,SCO0319,319499,-1,1,311,0.90,AGG,TCCCGTGCCCGGCCCCGGCC,CCCGGCCCCGGCC,10,"0,1,0,0","4,8,9,10"
85527,NC_003888.3,SCO0428,445611,1,1,342,0.75,CGG,GCGTTCACCGCGGTCCTGCG,CCGCGGTCCTGCG,10,"0,0,0","6,8,9"
60309,NC_003888.3,SCO0324,325238,-1,-1,23,0.95,TGG,CCGCGCCGCGCTGGCCGCCC,GCGCTGGCCGCCC,10,"1,1,0,1","4,6,7,9"


In [11]:
# Process the DataFrame to apply C-to-T mutations
mutated_sgrna_df = process_base_editing(filtered_sgrna_df_for_base_editing, 
                                        genes_to_KO_dict, 
                                        only_stop_codons = False, 
                                        editing_context= False)
mutated_sgrna_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,editing_context,editable_cytosines,mutations
4085,NC_003888.3,SCO0022,22156,-1,1,127,0.65,CGG,CACGAGAACCCCGAGCAGTG,ACCCCGAGCAGTG,-1,"0,0","9,10",V47I
4122,NC_003888.3,SCO0022,22156,-1,-1,270,0.65,CGG,GGCCAGGTTGGGCCAGGTTT,TTGGGCCAGGTTT,-1,0,4,Q92*
4123,NC_003888.3,SCO0022,22156,-1,-1,264,0.75,AGG,GAGTGGGGCCAGGTTGGGCC,GCCAGGTTGGGCC,-1,"1,0","9,10",Q92*
4096,NC_003888.3,SCO0022,22156,-1,1,275,0.70,TGG,CGCCGAAACCTGGCCCAACC,ACCTGGCCCAACC,-1,"0,0,0","4,9,10","G96N, G98S"
154973,NC_003888.3,SCO0736,779131,-1,-1,616,0.65,TGG,GGACCGCCGCCAAGAAGTAC,CGCCAAGAAGTAC,0,"0,0,1,0,1","4,5,7,8,10","T207I, A208V, A209V"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25476,NC_003888.3,SCO0154,144704,-1,-1,86,0.85,TGG,CGGCGCCGAGCAGGCCGAGC,GAGCAGGCCGAGC,10,"1,1,0","4,6,7",A31V
49461,NC_003888.3,SCO0281,272466,-1,1,224,0.85,AGG,CGGCGTCGCAGCGGGCCAGG,GCAGCGGGCCAGG,10,"1,0,1","4,7,9","C79Y, D80N, A81T"
58645,NC_003888.3,SCO0319,319499,-1,1,311,0.90,AGG,TCCCGTGCCCGGCCCCGGCC,CCCGGCCCCGGCC,10,"0,1,0,0","4,8,9,10","G108K, G110R"
85527,NC_003888.3,SCO0428,445611,1,1,342,0.75,CGG,GCGTTCACCGCGGTCCTGCG,CCGCGGTCCTGCG,10,"0,0,0","6,8,9",T117I


In [12]:
# Process the DataFrame to apply C-to-T mutations
mutated_sgrna_df = process_base_editing(filtered_sgrna_df_for_base_editing, 
                                        genes_to_KO_dict, 
                                        only_stop_codons = True, 
                                        editing_context= False)
mutated_sgrna_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,editing_context,editable_cytosines,mutations
78735,NC_003888.3,SCO0392,413247,1,1,0,0.55,CGG,ATGACACGATGCCGACTCTG,GATGCCGACTCTG,0,"0,0","5,7","T2I, R3*"
130942,NC_003888.3,SCO0622,662373,1,1,0,0.40,GGG,ATGCAACTTATGTCGGAGAA,TTATGTCGGAGAA,0,"1,0","4,7","Q2*, L3F"
39894,NC_003888.3,SCO0232,221414,-1,-1,0,0.60,GGG,ATGCGACTGCGCTGCTCAGA,TGCGCTGCTCAGA,0,"1,0,1","4,7,10","R2*, R4C"
48177,NC_003888.3,SCO0274,264764,1,1,0,0.55,CGG,ATGGCCCAGCTCGTAGAACT,AGCTCGTAGAACT,1,"1,0,0,1","5,6,7,10","A2V, Q3*, L4F"
10892,NC_003888.3,SCO0071,60069,-1,-1,0,0.55,AGG,ATGCAGATCAGCTTGGAGGC,TCAGCTTGGAGGC,0,"1,0","4,9",Q2*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101282,NC_003888.3,SCO0492,513989,-1,-1,9912,0.70,GGG,GCGCAGCAACTGCTCGCCGA,AACTGCTCGCCGA,2,"1,1,0","4,7,10","Q3306*, Q3307*"
101057,NC_003888.3,SCO0492,513989,-1,1,9997,0.75,GGG,CAGCCGCGTCCACAGTGCGG,GTCCACAGTGCGG,0,"1,0,1,0","4,5,7,10","W3336*, R3338Q"
101238,NC_003888.3,SCO0492,513989,-1,-1,10317,0.75,CGG,CTGGTGGTGCGACGCCCCGA,TGCGACGCCCCGA,0,1,10,R3443*
101235,NC_003888.3,SCO0492,513989,-1,-1,10365,0.60,CGG,GACCAGCAGGTCAAGATCCG,AGGTCAAGATCCG,4,"0,1","4,7","Q3457*, Q3458*"


In [13]:
# Filter the DataFrame to retain only up to 5 sgRNA sequences per locus_tag
filtered_df = mutated_sgrna_df.groupby('locus_tag').head(number_of_sgRNAs_per_group)
filtered_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,editing_context,editable_cytosines,mutations
78735,NC_003888.3,SCO0392,413247,1,1,0,0.55,CGG,ATGACACGATGCCGACTCTG,GATGCCGACTCTG,0,"0,0","5,7","T2I, R3*"
130942,NC_003888.3,SCO0622,662373,1,1,0,0.40,GGG,ATGCAACTTATGTCGGAGAA,TTATGTCGGAGAA,0,"1,0","4,7","Q2*, L3F"
39894,NC_003888.3,SCO0232,221414,-1,-1,0,0.60,GGG,ATGCGACTGCGCTGCTCAGA,TGCGCTGCTCAGA,0,"1,0,1","4,7,10","R2*, R4C"
48177,NC_003888.3,SCO0274,264764,1,1,0,0.55,CGG,ATGGCCCAGCTCGTAGAACT,AGCTCGTAGAACT,1,"1,0,0,1","5,6,7,10","A2V, Q3*, L4F"
10892,NC_003888.3,SCO0071,60069,-1,-1,0,0.55,AGG,ATGCAGATCAGCTTGGAGGC,TCAGCTTGGAGGC,0,"1,0","4,9",Q2*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114871,NC_003888.3,SCO0546,583523,-1,-1,627,0.65,CGG,GAGGTGCAGATCCTCGCCGA,AGATCCTCGCCGA,0,1,7,Q212*
125968,NC_003888.3,SCO0599,638801,1,1,644,0.80,CGG,CGGGCAGCGGCTGCAGGACG,CGGCTGCAGGACG,0,"1,1","5,8","Q217*, R218W"
205697,NC_003888.3,SCO0976,1028822,-1,1,671,0.75,AGG,CTCCCCACTCACCCGCCGGT,CTCACCCGCCGGT,0,"0,0,0,0,0","4,5,6,8,10","E228K, W229*, G230R"
130175,NC_003888.3,SCO0618,659143,1,-1,782,0.85,AGG,CGACCCACTCGCCCGCGGCC,CTCGCCCGCGGCC,0,"0,0,0,0,0","4,5,6,8,10","E265K, W266*, V267I"


## Output

In [14]:
# MAke oligoes
list_of_ssDNAs = make_ssDNA_oligos(filtered_df, upstream_ovh = up_homology,
                      downstream_ovh=dw_homology)


# cut plasmid
from Bio.Restriction import NcoI, NheI
linearized_plasmid = sorted(clean_plasmid.cut(NcoI,NheI ), key=lambda x: len(x), reverse=True)[0]

# assemble plasmid
sgRNA_vectors = assemble_plasmids_by_ssDNA_bridging(list_of_ssDNAs,linearized_plasmid)

In [15]:
# Constructing a meaningful name, ID, and description for the assembled plasmid using user input
targeting_info = []
for index, row in filtered_df.iterrows():
    formatted_str = f"pCRISPR-BEST_{row['locus_tag']}_p{row['sgrna_loc']}"
    targeting_info.append(formatted_str)

for i in range(len(sgRNA_vectors)):
    sgRNA_vectors[i].name = f'{targeting_info[i]}_#{i+1}'
    sgRNA_vectors[i].id = sgRNA_vectors[i].name  # Using the same value for ID as for name for simplicity
    sgRNA_vectors[i].description = f'Assembled plasmid targeting {", ".join(genes_to_KO)} for base-editing, assembled using StreptoCAD.'

# annotate plasmids
for plasmid in sgRNA_vectors: 
    annotate_plasmid_with_sgrnas(plasmid, filtered_df)

# PRINT
print_plasmids = False
if print_plasmids: 
    for vector in sgRNA_vectors: 
        vector.write(f"../../data/plasmids/sgRNA_plasmids_pCRISPR_cBEST/{vector.id}.gb")

In [16]:
integration_names = filtered_df.apply(lambda row: f"sgRNA_{row['locus_tag']}({row['sgrna_loc']})", axis=1).tolist()
plasmid_metadata_df = extract_metadata_to_dataframe(sgRNA_vectors,
                                                    clean_plasmid,
                                                    integration_names)

plasmid_metadata_df

,plasmid_name,date,original_plasmid,integration,size
0,pCRISPR-BEST_SCO0392_p0_#1,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0392(0),11817
1,pCRISPR-BEST_SCO0622_p0_#2,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0622(0),11817
2,pCRISPR-BEST_SCO0232_p0_#3,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0232(0),11817
3,pCRISPR-BEST_SCO0274_p0_#4,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0274(0),11817
4,pCRISPR-BEST_SCO0071_p0_#5,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0071(0),11817
...,...,...,...,...,...
938,pCRISPR-BEST_SCO0546_p627_#939,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0546(627),11817
939,pCRISPR-BEST_SCO0599_p644_#940,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0599(644),11817
940,pCRISPR-BEST_SCO0976_p671_#941,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0976(671),11817
941,pCRISPR-BEST_SCO0618_p782_#942,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0618(782),11817


# IDT primers

In [17]:
full_idt = primers_to_IDT(list_of_ssDNAs)
full_idt

,Name,Sequence,Concentration,Purification
0,SCO0392_loc_0,cggttggtaggatcgacggcATGACACGATGCCGACTCTGtatgtc...,25nm,STD
1,SCO0622_loc_0,cggttggtaggatcgacggcATGCAACTTATGTCGGAGAAtatgtc...,25nm,STD
2,SCO0232_loc_0,cggttggtaggatcgacggcATGCGACTGCGCTGCTCAGAtatgtc...,25nm,STD
3,SCO0274_loc_0,cggttggtaggatcgacggcATGGCCCAGCTCGTAGAACTtatgtc...,25nm,STD
4,SCO0071_loc_0,cggttggtaggatcgacggcATGCAGATCAGCTTGGAGGCtatgtc...,25nm,STD
...,...,...,...,...
938,SCO0546_loc_627,cggttggtaggatcgacggcGAGGTGCAGATCCTCGCCGAtatgtc...,25nm,STD
939,SCO0599_loc_644,cggttggtaggatcgacggcCGGGCAGCGGCTGCAGGACGtatgtc...,25nm,STD
940,SCO0976_loc_671,cggttggtaggatcgacggcCTCCCCACTCACCCGCCGGTtatgtc...,25nm,STD
941,SCO0618_loc_782,cggttggtaggatcgacggcCGACCCACTCGCCCGCGGCCtatgtc...,25nm,STD


## Folder with all the generated I/O

In [18]:
input_files = [
    {"name": "input_genome.gb", "content": genome},
    {"name": "input_plasmid.gb", "content": clean_plasmid}
]

output_files = [
    {"name": "cBEST_w_sgRNAs.gb", "content": sgRNA_vectors}, # LIST OF Dseqrecords
    {"name": "full_idt.csv", "content": full_idt},
    {"name": "mutated_sgrna_df.csv", "content": mutated_sgrna_df},
    {"name": "filtered_df.csv", "content": filtered_df},
    {"name": "plasmid_metadata_df.csv", "content": plasmid_metadata_df},

]

input_values = {
    "genes_to_knockout": genes_to_KO,
    "polymerase_settings": {
        "chosen_polymerase": chosen_polymerase,
        "melting_temperature": melting_temperature,
        "primer_concentration": primer_concentration,
        "primer_number_increment": primer_number_increment,
        "flanking_region_number": flanking_region_number
    },
    "filtering_metrics": {
        "gc_upper": gc_upper,
        "gc_lower": gc_lower,
        "off_target_seed": off_target_seed,
        "off_target_upper": off_target_upper,
        "cas_type": cas_type,
        "number_of_sgRNAs_per_group": number_of_sgRNAs_per_group
    },
    "overlapping_sequences": {
        "up_homology": str(up_homology),
        "dw_homology": str(dw_homology)
    }
}


# Paths to Markdown files
markdown_file_paths = [
    "../../protocols/conjugation_protcol.md",
    "../../protocols/single_target_crispr_plasmid_protcol.md"

]

# Data and time
timestamp = datetime.utcnow().isoformat()

# Create project directory structure
project_directory = ProjectDirectory(
    project_name=f"CRISPR_cBEST_workflow_{timestamp}",
    input_files=input_files,
    output_files=output_files,
    input_values=input_values,
    markdown_file_paths=markdown_file_paths
)


# DO You want to save the folder? 
save_zip_folder = False 

if save_zip_folder: 
    # Generate the project directory structure and get the zip content
    zip_content = project_directory.create_directory_structure(create_directories=False)

    # Save the zip file to disk (optional)
    with open("project_structure.zip", "wb") as f:
        f.write(zip_content)